In [115]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [125]:
function kalman_filter_graph()
    x_prev_add = AdditionNode()
    
    x_prior = datavar(:x_prior, Normal{Float64})
    add_1   = constvar(:add_1, 1.0)

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    
    add_x_and_noise = AdditionNode()
    
    x = simplerandomvar(:x)
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = datavar(:y, Float64)
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(x_prev_add)
    activate!(add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()
    
    beliefs = Vector{Normal{Float64}}(undef, N)
    updates = Channel{Tuple{Normal{Float64}, Float64}}(Inf) do ch
        while true
            update_data = take!(ch)
            update!(x_prior, update_data[1])
            update!(y, update_data[2])
        end
    end
    
    actor = sync(lambda(Tuple{Int, AbstractBelief},
        on_next = (d) -> begin
        
            index  = d[1]    
            belief = getdata(d[2])
                    
            if index < N
                push!(updates, (belief, data[index + 1]))
            else
                finish!(x_prior)
                finish!(y)
                close(updates)
            end
                
            beliefs[index] = belief
        end,
        on_error = (e) -> println(e)
    ), timeout = 10000)
    
    @async begin
        try
            subscribe!(belief(x) |> enumerate(), actor)
            push!(updates, (Normal(0.0, 1000.0), data[1]))
        catch err 
            println(err)
        end
    end
    
    wait(actor)
    
    return beliefs
end

kalman (generic function with 1 method)

In [126]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [129]:
@benchmark kalman($data)

BenchmarkTools.Trial: 
  memory estimate:  439.47 KiB
  allocs estimate:  11335
  --------------
  minimum time:     1.118 ms (0.00% GC)
  median time:      1.226 ms (0.00% GC)
  mean time:        1.387 ms (6.08% GC)
  maximum time:     110.626 ms (97.76% GC)
  --------------
  samples:          3592
  evals/sample:     1

In [128]:
@time kalman(data)

  0.001933 seconds (11.33 k allocations: 439.453 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-19.938566894038704, σ=14.140721622265264)
 Normal{Float64}(μ=-3.1798097440385398, σ=9.999500037496876)
 Normal{Float64}(μ=-7.750864094687809, σ=8.164693657357805)
 Normal{Float64}(μ=1.4803327077924162, σ=7.070891041799029)
 Normal{Float64}(μ=4.112374919968087, σ=6.3244288330249585)
 Normal{Float64}(μ=5.348537487795703, σ=5.773406469256952)
 Normal{Float64}(μ=2.5035912914804497, σ=5.34514847952991)
 Normal{Float64}(μ=4.785611970869732, σ=4.999937501171851)
 Normal{Float64}(μ=8.127714144117396, σ=4.713992830503184)
 Normal{Float64}(μ=10.830502570298275, σ=4.472091234310838)
 Normal{Float64}(μ=12.843593115797232, σ=4.263975563874187)
 Normal{Float64}(μ=12.738281899180004, σ=4.082448884373011)
 Normal{Float64}(μ=14.14824425410926, σ=3.922292531398713)
 ⋮
 Normal{Float64}(μ=588.3807007367329, σ=0.5827164478148905)
 Normal{Float64}(μ=589.3876290837466, σ=0.5822224110578259)
 Normal{Float64}(μ=590.389767829854, σ=0.5817296287317493)
 N

In [127]:
@time kalman(data)

  1.634474 seconds (3.27 M allocations: 172.233 MiB, 6.59% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-19.938566894038704, σ=14.140721622265264)
 Normal{Float64}(μ=-3.1798097440385398, σ=9.999500037496876)
 Normal{Float64}(μ=-7.750864094687809, σ=8.164693657357805)
 Normal{Float64}(μ=1.4803327077924162, σ=7.070891041799029)
 Normal{Float64}(μ=4.112374919968087, σ=6.3244288330249585)
 Normal{Float64}(μ=5.348537487795703, σ=5.773406469256952)
 Normal{Float64}(μ=2.5035912914804497, σ=5.34514847952991)
 Normal{Float64}(μ=4.785611970869732, σ=4.999937501171851)
 Normal{Float64}(μ=8.127714144117396, σ=4.713992830503184)
 Normal{Float64}(μ=10.830502570298275, σ=4.472091234310838)
 Normal{Float64}(μ=12.843593115797232, σ=4.263975563874187)
 Normal{Float64}(μ=12.738281899180004, σ=4.082448884373011)
 Normal{Float64}(μ=14.14824425410926, σ=3.922292531398713)
 ⋮
 Normal{Float64}(μ=588.3807007367329, σ=0.5827164478148905)
 Normal{Float64}(μ=589.3876290837466, σ=0.5822224110578259)
 Normal{Float64}(μ=590.389767829854, σ=0.5817296287317493)
 N